# Trabalho de Programação Linear aplicada ao problema de Cinemática Inversa de Braços Robóticos

Para rodar a simulação no Google Colab execute os comandos abaixo:

!pip install roboticstoolbox-python

!pip install ortools       

In [ ]:
!pip install cvxpy

https://github.com/petercorke/robotics-toolbox-python

https://petercorke.github.io/robotics-toolbox-python/index.html

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

In [ ]:
import roboticstoolbox as rtb
import spatialmath as sm
import numpy as np
from roboticstoolbox.backends.PyPlot import PyPlot

In [ ]:
# Implemente o modelo de Prog. Quad. dentro da função solve usando a biblioteca cvxpy
import cvxpy as cvx

def solve(Q, c, G, h, A, b):
    pass

In [ ]:
def build_equality_constraints(panda, effector_velocity):
    J = panda.jacobe(panda.q)
    # Gera a matriz estendida A:6x13 a partir da concatenação de J:6x7 e I:6x6
    A = np.c_[J, np.eye(6)]
    b = effector_velocity
    return A, b

In [ ]:
def build_quadratic_function(panda, current_effector_pose, target_effector_pose, joint_velocity, n):
    
    error = current_effector_pose.inv() * target_effector_pose

    e = np.sum(np.abs(np.r_[error.t, error.rpy() * np.pi / 180]))

    # Matriz de covariância Q:13x13
    Q = np.eye(n + 6)

    # Termo de ganho (lambda) para controle de velocidade das juntas
    Q[:n, :n] *= 0.01

    # Correlação do erro dados pela diferença entre as poses atual e final atrelada a velocidade da garra
    Q[n:, n:] = (1 / e) * np.eye(6)

    # Vetor de coeficientes c:13
    c = np.r_[-panda.jacobm(joint_velocity).reshape((n,)), np.zeros(6)]

    return Q, c

In [ ]:
def build_inequality_constraints(panda, n):
    A = np.zeros((n + 6, n + 6))
    b = np.zeros(n + 6)
    # Limites de velocidades da juntas para serem consideradas nas restrições
    A[:n, :n], b[:n] = panda.joint_velocity_damper(ps=0.05, pi=0.9, n=n)
    return A, b

In [ ]:
# Carrega um modelo do robô Panda e define seus ângulos de junta para a configuração de prontidão
panda = rtb.models.DH.Panda()
panda.q = panda.qr

n = 7

joint_velocity = np.zeros(7)

current_effector_pose = panda.fkine(panda.q)

# Define uma pose desejada para a garra em função da pose atual da garra
target_effector_pose = current_effector_pose * sm.SE3.Tx(0.2) * sm.SE3.Ty(0.2) * sm.SE3.Tz(0.45)

panda.plot(panda.q)

In [ ]:
panda.q

In [ ]:
env = PyPlot()
env.launch()
env.add(panda)

In [ ]:
# Simula o robô enquanto ele não alcança o objetivo
effector_arrived = False
while not effector_arrived:
    current_effector_pose = panda.fkine(panda.q)
    
    # Calcula a velocidade necessária da garra para ir em direção ao objetivo
    effector_velocity, effector_arrived = rtb.p_servo(current_effector_pose, target_effector_pose, 1)
    
    J = panda.jacobe(panda.q)
    
    A, b = build_equality_constraints(panda, effector_velocity)

    G, h = build_inequality_constraints(panda, n)

    Q, c = build_quadratic_function(panda, current_effector_pose, target_effector_pose, joint_velocity, n)

    joint_velocity = solve(Q, c, G, h, A, b)

    # Calcula as velocidades das juntas do Panda conforme Eq. 8.3 do livro do Peter Corke (pag. 234)
    joint_velocity = np.linalg.pinv(J) @ effector_velocity

    # Defina as velocidades das juntas do Panda
    panda.qd = joint_velocity
    
    env.step(0.05)